In [1]:
from google.colab import drive
drive.mount('/content/drive')
wdir = '/content/drive/MyDrive/Lord-of-the-Rings-SLM-main'
import os
os.chdir(wdir)

Mounted at /content/drive


In [2]:
import torch
from operator import itemgetter
from torch.utils.data import DataLoader
import random
import numpy as np
import math
import os
from update_utilities import update_utilities_class
import pickle

In [3]:
with open('saved_encoder_dict.pkl','rb') as f:
    encode_char = pickle.load(f)

with open('saved_decoder_dict.pkl','rb') as f:
    decode_char = pickle.load(f)

print(len(encode_char))
vocab_size = len(encode_char)

74


In [4]:
encode = lambda string: [encode_char[s] for s in string]
decode = lambda nums: ''.join([decode_char[n] for n in nums])

In [5]:
import Transformer
block_size=512
device='cpu'
transformer = Transformer.TransformerClass(vocab_size=vocab_size,emb_dim=512,n_layer=8,num_heads=8,block_size=block_size,
                               dropout_rate_attention=0.1,dropout_rate_ff=0.2,dropout_rate_pos_enc=0.1,
                               is_decoder = True, ff_multiplier = 4).to(device)

In [6]:
print(round(sum(p.numel() for p in transformer.parameters())/1e6,2), 'M parameters')

25.28 M parameters


In [7]:
transformer.load_state_dict(torch.load("base_line_GPT_SG stats/base_line_GPT_SG weights/base_line_GPT_SG_last.pth",

                                       # If you are loading the model on a GPU, remove the following param
                                       map_location=torch.device('cpu')))

<All keys matched successfully>

In [8]:
def generate(model,max_new_tokens,encoder,decoder,block_size=512):
    print(f"Please enter some text to start the text generation, ideally the length you type is between 20 and 50 words, model will generate {max_new_tokens} characters\n")
    user_input = input(f"Enter here: ")
    input_encode = torch.tensor(encoder(user_input))
    output = list(input_encode)
    if len(input_encode) < block_size:
        pad_length = block_size - len(input_encode)
        idx = torch.zeros((1,block_size),dtype=torch.long)
        idx[:,pad_length:] = input_encode
        #print(decoder(idx.squeeze().tolist()))
    else:
        idx = input_encode[-block_size:].unsqueeze(0)
        #print(decoder(idx.squeeze().tolist()))

    print_status = False
    print_index_start = len(output)
    print("\n\n")
    print("Generating..........................\n")
    sentence_end = encoder(".!?' ")




    for _ in range(max_new_tokens):
        if len(output) % 100 == 0:
            print_status = True
        if print_status and output[-1] in sentence_end:
            print_setence = decoder(output[print_index_start:])
            print_index_start = len(output)
            print_status=False
            print(print_setence + "\n")
        idx_cond = idx[:,-block_size:]
        logits = model(idx_cond) # B, T, vocab_size
        logits = logits[:,-1,:] # logits for the last character
        probs = torch.nn.functional.softmax(logits.squeeze(),dim=0)
        idx_next = torch.multinomial(probs,num_samples=1)
        output.append(idx_next.squeeze().item())
        idx = torch.cat((idx,idx_next.unsqueeze(0)),dim=1)





In [9]:
generate(transformer,2000,encoder = encode, decoder = decode,block_size=512)

Please enter some text to start the text generation, ideally the length you type is between 20 and 50 words, model will generate 2000 characters

Enter here: When Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday



Generating..........................

wing a hundred quietly. 'Yes,' said Gandalf. 'Bet how for Gourth that ar be bit now! a I am 

bearr our on- burst t trumb freelly. O Hob! Mr. Bill rapped it. He anothel! Holde me, ill, I real at 

us custo west being out quest, At and quick fruisined was such it spliked uny there! Nor cruwl, and 

clear troub! Smeagord. At nour quail, hoot Gimli's beginning to some its talking. Then dismay say probably 

a hobbit have climbed, but took the trees they did, and is surely folk. 'What a wright? Have 

he a part of minit, if any wander, allow! dwarves: when you honour went hailed to be thirsty! Small 

sharp with my home, and tide safe coming! Who are you hear your comand, the Redhor?' sa

In [10]:
a = torch.randint(70, (100,))
pad_length = 512 - len(a)
idx = torch.zeros((1,block_size),dtype=torch.long)
idx[:,pad_length:] = a
print(idx.shape)
b = transformer(idx)
print(b.shape)

torch.Size([1, 512])
torch.Size([1, 512, 74])


In [11]:
logits = b[:,-1,:].squeeze()
logits.shape

torch.Size([74])

In [12]:
logits

tensor([ 4.9992,  2.9258,  3.4528, -3.6262, -2.8265,  3.1062,  1.0266,  1.8133,
        -5.8319, -3.5426, -3.7662, -4.1765, -4.8530, -5.1152, -3.5505, -3.5268,
        -5.6295, -6.5682, -0.1368, -1.0152,  0.7477, -0.9428, -2.5783, -1.6628,
        -1.5492, -1.0850, -1.1269, -1.1531, -1.2569, -1.4066, -3.6206, -2.2207,
        -2.0257, -2.2312, -1.6317, -1.0564, -2.1495, -3.1191, -2.0759, -0.7071,
        -0.3503, -2.9251, -2.6559, -0.7180, -7.2718, -1.8576, -5.3768,  2.1980,
        -0.8541, -0.3581, -4.1498,  5.1302, -0.0544, -1.3491,  2.3043,  2.8625,
        -1.9094, -1.7690, -0.1984, -3.1668,  0.4004,  0.1983, -2.3763, -3.5921,
        -1.4407,  0.5923,  0.9673, -1.2431, -2.3722, -0.8788, -2.4983,  1.8963,
        -4.9447, -3.4274], grad_fn=<SqueezeBackward0>)

In [13]:
probs = torch.nn.functional.softmax(logits,dim=0)
probs

tensor([3.2021e-01, 4.0270e-02, 6.8212e-02, 5.7474e-05, 1.2788e-04, 4.8230e-02,
        6.0280e-03, 1.3239e-02, 6.3320e-06, 6.2487e-05, 4.9968e-05, 3.3151e-05,
        1.6853e-05, 1.2967e-05, 6.1993e-05, 6.3481e-05, 7.7526e-06, 3.0323e-06,
        1.8832e-03, 7.8241e-04, 4.5605e-03, 8.4111e-04, 1.6389e-04, 4.0940e-04,
        4.5869e-04, 7.2962e-04, 6.9972e-04, 6.8158e-04, 6.1439e-04, 5.2896e-04,
        5.7796e-05, 2.3436e-04, 2.8482e-04, 2.3190e-04, 4.2236e-04, 7.5081e-04,
        2.5164e-04, 9.5436e-05, 2.7088e-04, 1.0647e-03, 1.5211e-03, 1.1587e-04,
        1.5166e-04, 1.0532e-03, 1.5005e-06, 3.3694e-04, 9.9816e-06, 1.9449e-02,
        9.1912e-04, 1.5094e-03, 3.4046e-05, 3.6503e-01, 2.0449e-03, 5.6029e-04,
        2.1630e-02, 3.7798e-02, 3.1994e-04, 3.6816e-04, 1.7707e-03, 9.0989e-05,
        3.2227e-03, 2.6329e-03, 2.0058e-04, 5.9469e-05, 5.1125e-04, 3.9044e-03,
        5.6807e-03, 6.2291e-04, 2.0140e-04, 8.9670e-04, 1.7755e-04, 1.4384e-02,
        1.5376e-05, 7.0113e-05], grad_fn

In [14]:

d = torch.multinomial(probs,num_samples=1).unsqueeze(0)
print(d.shape)
print(d)

torch.Size([1, 1])
tensor([[2]])


In [15]:
d.squeeze().item()

2